In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense

#copypasted -> for gpu usage
import tensorflow as tf
from keras.utils.training_utils import multi_gpu_model
from tensorflow.python.client import device_lib 

In [2]:
# IMPORTANT: Tells tf to not occupy a specific amount of memory
from keras.backend.tensorflow_backend import set_session  
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU  
sess = tf.Session(config=config)  
set_session(sess)  # set this TensorFlow session as the default session for Keras.


# getting the number of GPUs 
def get_available_gpus():
   local_device_protos = device_lib.list_local_devices()
   return [x.name for x in local_device_protos if x.device_type    == 'GPU']
num_gpu = len(get_available_gpus())
print('Amount of GPUs available: %s' % num_gpu)

Amount of GPUs available: 4


In [3]:
#Get data and make it usable
df = pd.read_csv('//datc//opschaler//combined_gas_smart_weather_dfs//processed//all_dwellings_combined_hour.csv', delimiter='\t', parse_dates=['datetime'])
df = df.set_index(['datetime'])
df = df.dropna()
df['gasPower'] = df['gasPower']*10
df['T-1'] = df['T'].shift(1)
df_d = df.resample('1D').mean() #Resampling to a day
df_d = df_d.dropna()


In [4]:
#Add dummies!
df_d['hour'] = df_d.index.hour #create dummy variable (hour)
df_d['dayofweek'] = df_d.index.dayofweek #Create dummy variable (day of the week)
columns_to_cat = ['hour', 'dayofweek']
df_d[columns_to_cat] = df_d[columns_to_cat].astype('category') # change datetypes->category
df_d = pd.get_dummies(df_d, columns=columns_to_cat)


In [8]:
#Split, originally thought about using the train_test_split function but it seems to give some errors
X = df_d[['T','T-1','dayofweek_0','dayofweek_1','dayofweek_2','dayofweek_3','dayofweek_4','dayofweek_5','dayofweek_6']]
y = df_d[['gasPower']]
labelencoder_y_1 = LabelEncoder()
y = labelencoder_y_1.fit_transform(y)

test_size=0.3
split_index = int(df_d.shape[0]*test_size)

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

scalerX = StandardScaler(with_mean=True, with_std=True).fit(X_train) #Normalization! :D

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [9]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
forecast = Sequential()

#first layer with the input data
forecast.add(Dense(output_dim =16, init = 'uniform', activation = 'relu', input_dim = 9))
#second layer 
forecast.add(Dense(output_dim = 16, init = 'uniform', activation = 'relu'))
#output layer
forecast.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=9, units=16, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=16, kernel_initializer="uniform")`
  
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [11]:
forecast_model = multi_gpu_model(forecast, gpus=num_gpu)

In [12]:
forecast_model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

In [13]:
y_train = (y_train-min(y_train))/(max(y_train)-min(y_train))

forecast_model.fit(X_train, y_train, batch_size = 10,epochs = 100);

Epoch 1/100
83/83 [==============================] - 2s 26ms/step - loss: 0.0669 - acc: 0.0120
Epoch 2/100
83/83 [==============================] - 0s 394us/step - loss: 0.0658 - acc: 0.0120
Epoch 3/100
83/83 [==============================] - 0s 361us/step - loss: 0.0645 - acc: 0.0120
Epoch 4/100
83/83 [==============================] - 0s 427us/step - loss: 0.0628 - acc: 0.0120
Epoch 5/100
83/83 [==============================] - 0s 370us/step - loss: 0.0604 - acc: 0.0120
Epoch 6/100
83/83 [==============================] - 0s 372us/step - loss: 0.0574 - acc: 0.0120
Epoch 7/100
83/83 [==============================] - 0s 1ms/step - loss: 0.0532 - acc: 0.0120
Epoch 8/100
83/83 [==============================] - 0s 703us/step - loss: 0.0484 - acc: 0.0120
Epoch 9/100
83/83 [==============================] - 0s 556us/step - loss: 0.0432 - acc: 0.0120
Epoch 10/100
83/83 [==============================] - 0s 369us/step - loss: 0.0370 - acc: 0.0120
Epoch 11/100
83/83 [======================

In [15]:
y_pred = forecast_model.predict(X_test)